In [15]:
import tensorflow as tf
from scipy.misc import imread
import numpy as np
import os
from PIL import Image
import itertools
import time

In [16]:
classes = ["Benign", "InSitu", "Invasive", "Normal"]
dataset_path = os.path.join(os.path.pardir, 'dataset')
N_c = 50
N = N_c*4
width = 2048
height = 1536
rgb_channel = 3

dataset = np.ndarray([N, height, width, rgb_channel])

In [17]:
start_time = time.time()
for i, class_name in enumerate(classes):
    path = os.path.join(dataset_path, class_name)
    files = os.listdir(path)
    for j, img_name in enumerate(files):
        if j < N_c:
            im = Image.open(path + "/" + img_name)
            imarray = np.array(im)
            dataset[i*N_c + j] = imarray
    print(path + "/*", "... loaded")
end_time = time.time()
run_time = end_time-start_time
print(run_time)

../dataset/Benign/* ... loaded
../dataset/InSitu/* ... loaded
../dataset/Invasive/* ... loaded
../dataset/Normal/* ... loaded
26.396467208862305


In [18]:
'''
start_time = time.time()
np.save("dataset.npy", dataset)
end_time = time.time()
run_time = end_time-start_time
print(run_time)

start_time = time.time()
dataset = np.load("dataset.npy")
end_time = time.time()
run_time = end_time-start_time
print(run_time)
'''

'\nstart_time = time.time()\nnp.save("dataset.npy", dataset)\nend_time = time.time()\nrun_time = end_time-start_time\nprint(run_time)\n\nstart_time = time.time()\ndataset = np.load("dataset.npy")\nend_time = time.time()\nrun_time = end_time-start_time\nprint(run_time)\n'

In [19]:
#np.random.shuffle(dataset)
index_permutation = np.array(range(N))
np.random.shuffle(index_permutation)
print(index_permutation)

[ 67 191   0  40 189 146 197  69 104  44  23 173  29 126 182  74   9  52
  34 162  24  48 175 183 127  50 102 198 148 141 142 151 128 122  16 135
 125  53  14 177 134  66  49  71  15  19 120  56 110  59 165  47  79  45
  91 111  18 161  80  28  85 147 109 103 187  68 166 154 164 185 136  78
  46   8  54  75  25 143 124 172  12  89  87 149 139 145 158  36  95 159
  99  92 117  55 193 119  11 114 199 133  86 152  27 181 180  20 179  21
 168  84  35 196 194  98 107 163 105  64  32 140 129  90  62  57  97   6
  37   4  63   5 170  61 155 144  73 174 192  82  81  42 132 167 123  60
  70  41 137 118  39 188 178 131  76 176 113  38 116  10 156  17  22  65
 100 130  30 195 121   3   7 153  83 169  43 186 101  93 190  26  88 150
 160 106 138   2  31 157  58  77 115   1 171  13  51 112  33  96 184 108
  94  72]


In [20]:
n_training = int(.8 * N)
n_validation = int(.1 * N)
n_test = int(.1 * N)

training_set = dataset[index_permutation[0:n_training]]
validation_set = dataset[index_permutation[n_training:n_training+n_validation]]
test_set = dataset[index_permutation[n_training+n_validation:n_training+n_validation+n_test]]

print(training_set.shape)
print(validation_set.shape)
print(test_set.shape)

MemoryError: 